#### Imports

In [ ]:
import torch
import numpy
import math
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import sys
import os

sys.path.insert(0, os.path.abspath('../pytorch-physics/'))
from boid import Flock

#### Visualizations

In [ ]:
def visualize_boids(list_of_birds_pos, list_of_birds_vel, box_top):
    list_of_birds_pos = [list_of_birds_pos[i].cpu().numpy() for i in range(len(list_of_birds_pos))]
    list_of_birds_vel = [list_of_birds_vel[i].cpu().numpy() for i in range(len(list_of_birds_vel))]
    
    offset = math.floor(box_top / 50)
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xlim(offset, box_top - offset)
    ax.set_ylim(offset, box_top - offset)
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    
    initial_positions = list_of_birds_pos[0]
    initial_velocities = list_of_birds_vel[0]
    
    scatter = ax.scatter([], [], c='orange', s=10)
    
    quiver = ax.quiver(initial_positions[:, 0], initial_positions[:, 1], 
                       initial_velocities[:, 0], initial_velocities[:, 1], 
                       angles='xy', scale_units='xy', scale=1, color='blue')
    
    def update(frame):
        positions = list_of_birds_pos[frame]
        velocities = list_of_birds_vel[frame]
    
        scatter.set_offsets(positions)
    
        quiver.set_offsets(positions)
        quiver.set_UVC(velocities[:, 0], velocities[:, 1])
    
        return scatter, quiver
    
    ani = FuncAnimation(fig, update, frames=len(list_of_birds_pos), interval=50, blit=True)
    plt.close(fig)
    return HTML(ani.to_html5_video())

#### Config-Values

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
N = 400
box_top = 30

flock_args = {
    'D': 2,
    'N': N,
    'box_top': box_top,
    'pass_through_edges': True,
    'bouncy_edges': False,
    'device': device,
}

boid_args = {
    'init_speed': None,
    'min_speed': 0.02,
    'max_speed': 1,
    'max_acc': 0.1,
    
    'view_radius': 3,
    'view_angle': None,
    
    'avoid_radius': 5,
    'avoid_view': True,
    
    'sep_factor': 0.05,    # avoidfactor
    'align_factor': 0.05,  # matchingfactor
    'cohe_factor': 0.0005,  # centeringfactor
    'bias_factor': 0.005,
    'edge_factor': 0.05,
    
    'is_debug': False
}

#### Simulation and Visualization

In [ ]:
flock = Flock(
    **flock_args,
    **boid_args
)

list_of_birds_pos = []
list_of_birds_vel = []
iterations = 150

for _ in range(iterations):
    flock.update()
    list_of_birds_pos.append(flock.pos)
    list_of_birds_vel.append(flock.vel)

In [ ]:
visualize_boids(list_of_birds_pos, list_of_birds_vel, box_top)